In [23]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt

In [24]:
data = pd.read_csv('data_400_5000',sep = ',', index_col= 'Unnamed: 0')
data.head(3)

,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,-58.508839,...,40.0,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,-58.522982,...,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...
4,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3435548.0,"-38.0026256,-57.5494468",-38.002626,-57.549447,...,35.0,1828.571429,1828.571429,NaN,NaN,NaN,http://www.properati.com.ar/15bok_venta_depart...,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...,https://thumbs4.properati.com/5/xrRqlNcSI_vs-f...


In [25]:
# selección de barrios con similares características
data_state_mask = data.state_name == 'Capital Federal'
data_state = data[data_state_mask]
data_state.head()


,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,-58.508839,...,40.0,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,-58.522982,...,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...
7,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,"-34.5598729,-58.443362",-34.559873,-58.443362,...,40.0,3066.666667,3450.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bot_venta_depart...,EXCELENTE MONOAMBIENTE A ESTRENAR AMPLIO SUPER...,JOSE HERNANDEZ 1400 MONOAMBIENTE ESTRENAR CAT...,https://thumbs4.properati.com/1/IHxARynlr8sPEW...
8,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,"-34.5598729,-58.443362",-34.559873,-58.443362,...,60.0,3000.000000,3250.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bou_venta_depart...,EXCELENTE DOS AMBIENTES ESTRENAR AMPLIO SUPER...,"JOSE HERNANDEZ 1400 DOS AMBIENTES ESTRENAR ,...",https://thumbs4.properati.com/2/J3zOjgaFHrkvnv...
13,sell,apartment,Palermo Soho,|Argentina|Capital Federal|Palermo|Palermo Soho|,Argentina,Capital Federal,3430234.0,NaN,NaN,NaN,...,30.0,2234.000000,3723.333333,NaN,1.0,NaN,http://www.properati.com.ar/15bp9_venta_depart...,Torre I Mondrian. 3 ambientes con terraza y d...,Vitraux Palermo,https://thumbs4.properati.com/2/ZTUjkRzTz6YvcU...


In [26]:
data_state.pivot_table(index= 'place_name', columns= 'property_type', aggfunc= {'operation': 'count'})

operation                       
property_type               PH apartment  house store
place_name                                           
Abasto                    13.0      62.0    3.0   7.0
Agronomía                  3.0      20.0    1.0   1.0
Almagro                   54.0     707.0   21.0  22.0
Balvanera                 26.0     415.0    2.0  27.0
Barracas                  36.0     274.0   34.0  10.0
Barrio Norte               9.0     806.0    6.0  36.0
Belgrano                  34.0    1913.0   80.0  34.0
Boca                       8.0     141.0   13.0   5.0
Boedo                     25.0     310.0   23.0  17.0
Caballito                 57.0    1550.0   41.0  23.0
Capital Federal           26.0     332.0  369.0  42.0
Catalinas                  NaN       1.0    NaN   NaN
Centro / Microcentro       NaN     143.0    1.0  40.0
Chacarita                 16.0     153.0    5.0   4.0
Coghlan                   16.0     166.0   10.0   3.0
Colegiales                24.0     208.0   17.0   7.0
Congreso                   2.0     207.0    1.0  12.0
Constitución              12.0     136.0    3.0  10.0
Flores                    63.0     839.0   56.0  49.0
Floresta                  49.0     191.0   38.0   9.0
Las Cañitas                2.0     118.0    2.0   1.0
Liniers                   52.0     160.0   28.0   8.0
Mataderos                 80.0     106.0   86.0  14.0
Monserrat                 14.0     274.0    4.0  36.0
Monte Castro              13.0      28.0    4.0   1.0
Nuñez                     18.0     450.0   26.0  17.0
Once                       1.0     105.0    NaN  10.0
Palermo                   78.0    1691.0   36.0  54.0
Palermo Chico              1.0     141.0    6.0   NaN
Palermo Hollywood         14.0     392.0    8.0  18.0
Palermo Soho               8.0     271.0    7.0   8.0
Palermo Viejo              2.0      26.0    5.0   3.0
Parque Avellaneda         10.0      14.0    6.0   NaN
Parque Centenario          6.0      54.0    NaN   1.0
Parque Chacabuco          18.0      49.0   19.0   2.0
Parque Chas                1.0      10.0    2.0   2.0
Parque Patricios          24.0     171.0   14.0  18.0
Paternal                  22.0      69.0   10.0   4.0
Pompeya                    7.0      10.0   10.0   4.0
Puerto Madero              NaN     184.0    3.0   4.0
Recoleta                   6.0    1073.0    2.0  49.0
Retiro                     NaN     155.0    1.0  12.0
Saavedra                  32.0     337.0   30.0   7.0
San Cristobal             24.0     206.0   11.0  15.0
San Nicolás                4.0     106.0    NaN  10.0
San Telmo                 24.0     916.0    9.0  46.0
Tribunales                 2.0      55.0    NaN  16.0
Velez Sarsfield           11.0       9.0    6.0   NaN
Versalles                  3.0       9.0   14.0   NaN
Villa Crespo              66.0     873.0   19.0  57.0
Villa Devoto              38.0     173.0   43.0   7.0
Villa General Mitre        6.0      23.0   10.0   NaN
Villa Lugano              31.0      56.0   37.0   3.0
Villa Luro                25.0     253.0   14.0   4.0
Villa Ortuzar              5.0      63.0    2.0   6.0
Villa Pueyrredón          22.0      71.0   22.0   NaN
Villa Real                18.0       4.0    4.0   1.0
Villa Riachuelo            1.0       NaN    2.0   NaN
Villa Santa Rita           7.0      14.0    6.0   NaN
Villa Soldati              1.0       3.0    1.0   1.0
Villa Urquiza             63.0     970.0   40.0  11.0
Villa del Parque          30.0     279.0   27.0   7.0

In [27]:
data.shape

(62869, 25)

In [28]:
pd.set_option("display.max_rows", None)
data_state['place_name'].value_counts()

Belgrano                2061
Palermo                 1859
Caballito               1671
Recoleta                1130
Villa Urquiza           1084
Villa Crespo            1015
Flores                  1007
San Telmo                995
Barrio Norte             857
Almagro                  804
Capital Federal          769
Nuñez                    511
Balvanera                470
Palermo Hollywood        432
Saavedra                 406
Boedo                    375
Barracas                 354
Villa del Parque         343
Monserrat                328
Villa Luro               296
Palermo Soho             294
Floresta                 287
Mataderos                286
Villa Devoto             261
Colegiales               256
San Cristobal            256
Liniers                  248
Parque Patricios         227
Congreso                 222
Coghlan                  195
Puerto Madero            191
Centro / Microcentro     184
Chacarita                178
Retiro                   168
Boca          

A analizar:
rooms - cantidad de ambientes
amenities
tipo de propiedad
barrio

In [29]:
# data.groupby(['place_name','property_type'])['operation'].count()
data.pivot_table(index= 'place_name', columns= 'property_type', aggfunc= {'operation': 'count'})


operation                        
property_type                                PH apartment   house store
place_name                                                             
 Country Maschwitz Club                     NaN       NaN     2.0   NaN
 los alamos                                 NaN       1.0     NaN   NaN
Abasto                                     13.0      62.0     3.0   7.0
Abril Club de Campo                         NaN       NaN     4.0   NaN
Acacias Blancas                             NaN       NaN    21.0   NaN
Acassuso                                    6.0      61.0    98.0   5.0
Adrogué                                    11.0     110.0   229.0  17.0
Aeropuerto Internacional Ezeiza             NaN       NaN     1.0   NaN
Agronomía                                   3.0      20.0     1.0   1.0
Agua de Oro                                 NaN       NaN     1.0   NaN
Aguas Verdes                                1.0       1.0     1.0   NaN
Albanueva Barrio Cerrado                    NaN      11.0    11.0   NaN
Aldo Bonzi                                  NaN       NaN     2.0   NaN
Alejandro Korn                              NaN       1.0     3.0   NaN
Alem                                        NaN       2.0     NaN   NaN
Almagro                                    54.0     707.0    21.0  22.0
Almirante Brown                             1.0       6.0     7.0   1.0
Altamira                                    NaN       NaN    11.0   NaN
Alto Los Cardales                           NaN       NaN     1.0   NaN
Altos de Hudson II                          NaN       5.0     5.0   NaN
Altos de Manzanares 1 y 2                   NaN       NaN     2.0   NaN
Altos de Matheu                             NaN       NaN     2.0   NaN
Altos del Golf                              NaN       NaN     3.0   NaN
Altos del Pilar                             NaN       NaN     6.0   NaN
Alvear                                      NaN       NaN     1.0   NaN
Apóstoles                                   NaN       NaN     1.0   NaN
Arequito                                    NaN       1.0     NaN   NaN
Armenia Country Club                        NaN       4.0     1.0   NaN
Avellaneda                                 14.0     234.0    36.0  15.0
Azul                                        NaN       1.0     3.0   NaN
Bahía Blanca                                5.0     136.0    74.0   4.0
Bahía del Sol                               NaN       NaN     3.0   NaN
Balcarce                                    1.0       1.0    16.0   NaN
Balneario San Cayetano                      NaN       NaN     1.0   NaN
Balvanera                                  26.0     415.0     2.0  27.0
Banfield                                   11.0     256.0   149.0  25.0
Barbarita, Barrio Cerrado                   NaN       NaN    10.0   NaN
Barracas                                   36.0     274.0    34.0  10.0
Barrancas de San Jose                       NaN       NaN     8.0   NaN
Barrancas de Santa María                    NaN       NaN     5.0   NaN
Barrio Acacias                              NaN       NaN     3.0   NaN
Barrio Alto Camet                           NaN       NaN     1.0   NaN
Barrio Araucarias                           NaN       NaN     4.0   NaN
Barrio Barrancas del Lago                   NaN       NaN     4.0   NaN
Barrio Cabos del Lago                       NaN       NaN     2.0   NaN
Barrio Camet                                NaN       1.0     NaN   NaN
Barrio Cerrado "Ayres Plaza"                NaN       NaN    12.0   NaN
Barrio Cerrado "Buenos Aires Village"       NaN       NaN     2.0   NaN
Barrio Cerrado "Isla del Sol"               NaN       NaN     1.0   NaN
Barrio Cerrado "La Candelaria"              NaN       NaN     1.0   NaN
Barrio Cerrado "La Cautiva del Pilar"       NaN       2.0     1.0   NaN
Barrio Cerrado "La Damasia"                 NaN       NaN     1.0   NaN
Barrio Cerrado "La Delfina"                 NaN       NaN    15.0   NaN
Barrio

In [30]:
data_apparment = data.loc[data.property_type == 'apartment']
data_apparment.head()

,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,-58.522982,...,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...
4,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3435548.0,"-38.0026256,-57.5494468",-38.002626,-57.549447,...,35.0,1828.571429,1828.571429,NaN,NaN,NaN,http://www.properati.com.ar/15bok_venta_depart...,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...,https://thumbs4.properati.com/5/xrRqlNcSI_vs-f...
7,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,"-34.5598729,-58.443362",-34.559873,-58.443362,...,40.0,3066.666667,3450.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bot_venta_depart...,EXCELENTE MONOAMBIENTE A ESTRENAR AMPLIO SUPER...,JOSE HERNANDEZ 1400 MONOAMBIENTE ESTRENAR CAT...,https://thumbs4.properati.com/1/IHxARynlr8sPEW...
8,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,"-34.5598729,-58.443362",-34.559873,-58.443362,...,60.0,3000.000000,3250.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bou_venta_depart...,EXCELENTE DOS AMBIENTES ESTRENAR AMPLIO SUPER...,"JOSE HERNANDEZ 1400 DOS AMBIENTES ESTRENAR ,...",https://thumbs4.properati.com/2/J3zOjgaFHrkvnv...
12,sell,apartment,Martínez,|Argentina|Bs.As. G.B.A. Zona Norte|San Isidro...,Argentina,Bs.As. G.B.A. Zona Norte,3430813.0,"-34.4860195,-58.5038139",-34.486019,-58.503814,...,36.0,2948.717949,3194.444444,NaN,NaN,NaN,http://www.properati.com.ar/15bp4_venta_depart...,EXCELENTE DEPARTAMENTO 2 AMBIENTES CONTRAFRENT...,EXCELENTE DEPARTAMENTO 2 AMBIENTES CONTRAFRENT...,https://thumbs4.properati.com/6/wgSui4rTYqPncO...


In [31]:
data_apparment.shape

(41117, 25)

In [32]:
#busqueda de amenities en dataset final

#match amenities
pattern_amenities = "balcon|balcón"#|cochera|parrilla|pileta|amoblado|lavadero|patio|terraza|jardin|jardín"
match_pattern_amenities = data_apparment.description.str.contains(pattern_amenities, regex= True)
print('Coincidencias amenities ' + str(match_pattern_amenities.sum()))
mask_match_pattern_amenities_notnull = match_pattern_amenities.notnull()



Coincidencias amenities 15469


In [33]:
#incorporación de columna 'amenities' al data final
data_apparment.loc[mask_match_pattern_amenities_notnull, "amenities"] = match_pattern_amenities[mask_match_pattern_amenities_notnull].astype(int)

c:\Users\Irma\anaconda3\envs\dhdsblend2021\lib\site-packages\pandas\core\indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
c:\Users\Irma\anaconda3\envs\dhdsblend2021\lib\site-packages\pandas\core\indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [34]:

pattern_balcon = "balcón|balcon"

pattern_balcon_regex = re.compile(pattern_balcon)
address_series = data.Address
resultado_nombres = address_series.apply(lambda x: pattern_nombre_regex.search(x))
#print(type(resultado))
print(resultado_nombres)


AttributeError: 'DataFrame' object has no attribute 'Address'

In [ ]:
calles_match = resultado_nombres.apply(lambda x: x if x is None else x.group('nombre_calle'))
calles_match

In [ ]:
description_date_series = data_apparment['description']
description_date_series

7         EXCELENTE MONOAMBIENTE A ESTRENAR AMPLIO SUPER...
8         EXCELENTE DOS AMBIENTES  ESTRENAR AMPLIO SUPER...
19        Excelente semipiso al contra frente en Bulnes ...
21        EXCELENTE ZONA, MULTIPLES MEDIOS DE TRANSPORTE...
60        ESPECTACULAR DEPARTAMENTO DE 2 AMBIENTES CON C...
                                ...                        
121113    Amplio 3 ambientes en pleno Palermo, a una cua...
121146    VENTA PISO CON COCHERA EN PALERMOMuy buena ubi...
121157    Torre de 16º pisos ubicado en pleno centro com...
121160    Edificio en esquina de muy buena categoría de ...
121209    Edificio en esquina de muy buena categoría de ...
Name: description, Length: 4054, dtype: object

In [ ]:
amenities_pattern = "(P<balcon>(balcon|balcón))"
amenities_pattern_regex = re.compile(amenities_pattern)

data_amenities_match = description_date_series.apply(lambda x: x if x is np.NaN else amenities_pattern_regex.search(x))

mask_data_amenities_match_notnull = data_amenities_match.notnull()

data_apparment.loc[mask_data_amenities_match_notnull, "amenities_1"] = data_amenities_match[mask_data_amenities_match_notnull].apply(lambda x: x.group('balcon'))



c:\Users\Irma\anaconda3\envs\dhdsblend2021\lib\site-packages\pandas\core\indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [ ]:
data_amenities_match

7         None
8         None
19        None
21        None
60        None
          ... 
121113    None
121146    None
121157    None
121160    None
121209    None
Name: description, Length: 4054, dtype: object

In [ ]:
data_apparment['amenities_1']

7         NaN
8         NaN
19        NaN
21        NaN
60        NaN
         ... 
121113    NaN
121146    NaN
121157    NaN
121160    NaN
121209    NaN
Name: amenities_1, Length: 4054, dtype: object